In [ ]:
import csv
import os
from datetime import datetime
import json
import psycopg2
import sys
sys.path.insert(0,'..')

from scripts.database import get_conn_pool, execute, fetch_all
from scripts.routing import route_nogo, route_standard

In [ ]:
# connect to postgis database
pool = get_conn_pool({
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'postgres',
    'database': 'xlab-routing'
})

In [ ]:
# set start and end points

# START COORDINATES
start_lat = -1.271346
start_long = 36.751293

# END COORDINATES
end_lat = -1.190509
end_long = 36.726833

In [ ]:
# set nogo layers
nogo_layer_list = ['hgis_admin3', 'hgis_bridges_tunnels']

# set nogo layer queries
nogo_layer_query_list = ["hgis_admin3.adm3_name = 'Kitisuru'", 
                         "hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811"]

# set nogo point layer buffer distance (default is .0005 in degrees or 50 meters) 
nogo_point_buffer = .0005

# set nogo polyline layer buffer distance (default is .0001 in degrees or 10 meters)
nogo_line_buffer = .0001

# set nogo cost percentage weight (default is 100)
nogo_cost = 100


In [ ]:
# get route that avoids nogo geometry
route = route_nogo(nogo_layer_list, start_long, start_lat, end_long, end_lat, nogo_layer_query_list, 
                   nogo_point_buffer, nogo_line_buffer, nogo_cost)
wkt = route[0]
length = route[1]


In [ ]:
from ipyleaflet import Map, WKTLayer
import re

m = Map(center=[start_lat, end_long], zoom=12)
wlayer = WKTLayer(
    wkt_string=wkt,
    hover_style={"fillColor": "red"},
)
m.add_layer(wlayer)

m